## Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [1]:
import random

N = 1000000
A = [random.randint(0, 1000) for i in range(N)]
B = [a + 100 for a in A]

average_B = sum(B) / N
print("Среднее значение массива B:", average_B)


Среднее значение массива B: 600.207105


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
import pandas as pd
import numpy as np

# Создание таблицы с 2 млн строк и 4 столбцами со случайными числами
df = pd.DataFrame(np.random.randint(0, 1000000, size=(2000000, 4)), columns=['col1', 'col2', 'col3', 'col4'])

# Добавление столбца key со случайными английскими буквами
df['key'] = np.random.choice([chr(i) for i in range(65, 91)], size=2000000)

# Выборка подмножества строк, для которых в столбце key указаны первые 5 английских букв
subset_df = df[df['key'].isin(['A', 'B', 'C', 'D', 'E'])]

# Вывод результата
print(subset_df)


           col1    col2    col3    col4 key
3        493114  403401   40941   17101   C
6          3231  451998  435449  310679   A
12       300641  783223  745546  290053   E
19       799584  851350  855770  479994   D
24       722081  404933   25310  406996   C
...         ...     ...     ...     ...  ..
1999958  734336  407712  205864  843334   C
1999961  977978  341094  274168  561129   C
1999967  704184  868257  609709  507230   B
1999976  983261  156300  764865  996826   C
1999984  778255  806429  124493  159575   B

[384341 rows x 5 columns]


## Лабораторная работа 3

In [2]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.9/661.9 kB 16.4 MB/s eta 0:00:00


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [6]:
import pandas as pd

# Загрузка данных из файлов
recipes = pd.read_csv('recipes_sample.csv', delimiter=',', index_col='id')
reviews = pd.read_csv('reviews_sample.csv', delimiter=',', index_col='user_id')

# Приведение столбцов к нужным типам
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
reviews['date'] = pd.to_datetime(reviews['date'])
reviews.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

recipes.head()

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [7]:
reviews.head()

,id,recipe_id,date,rating,review
user_id,,,,,
21752,370476,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
431813,624300,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
400708,187037,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
2001852463,706134,404716,2017-12-11,5,These are a favorite for the holidays and so e...
95810,312179,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [23]:
#A

import time

def mean_rating_a(reviews):
    total_rating = 0
    count = 0
    for _, row in reviews.iterrows():
        if row['date'].year == 2010:
            total_rating += row['rating']
            count += 1
    return total_rating / count if count > 0 else 0

start_time = time.time()
mean_a = mean_rating_a(reviews)
print("Средний рейтинг по методу А:", mean_a)
print("Время выполнения метода А:", time.time() - start_time)

Средний рейтинг по методу А: 4.4544402182900615
Время выполнения метода А: 6.66293740272522


In [25]:
#Б

def mean_rating_b(reviews):
    total_rating = 0
    count = 0
    for _, row in reviews_2010.iterrows():
        total_rating += row['rating']
        count += 1
    return total_rating / count if count > 0 else 0

reviews_2010 = reviews[reviews['date'].dt.year == 2010]

start_time = time.time()
mean_b = mean_rating_b(reviews_2010)
print("Средний рейтинг по методу Б:", mean_b)
print("Время выполнения метода Б:", time.time() - start_time)

Средний рейтинг по методу Б: 4.4544402182900615
Время выполнения метода Б: 0.5057389736175537


In [26]:
#В

start_time = time.time()
mean_c = reviews[reviews['date'].dt.year == 2010]['rating'].mean()
print("Средний рейтинг по методу В:", mean_c)
print("Время выполнения метода В:", time.time() - start_time)


Средний рейтинг по методу В: 4.4544402182900615
Время выполнения метода В: 0.02868199348449707


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [27]:
!pip install line_profiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from line_profiler import LineProfiler

# определяем функции для профилирования
def mean_rating_a(reviews):
    total_rating = 0
    count = 0
    for _, row in reviews.iterrows():
        if row['date'].year == 2010:
            total_rating += row['rating']
            count += 1
    return total_rating / count if count > 0 else 0

def mean_rating_b(reviews):
    total_rating = 0
    count = 0
    for _, row in reviews_2010.iterrows():
        total_rating += row['rating']
        count += 1
    return total_rating / count if count > 0 else 0

def func_c():
    return reviews_2010['rating'].mean()

# создаем экземпляр LineProfiler
profiler = LineProfiler()

profiler.add_function(mean_rating_a)
profiler.add_function(mean_rating_b)
profiler.add_function(func_c)

#запускаем профилирование
profiler.enable()

#вызываем функции для подсчета среднего значения rating из таблицы reviews
mean_rating_a(reviews)
mean_rating_b(reviews_2010)
func_c()

#останавливаем профилирование
profiler.disable()

#выводим результаты профилирования
profiler.print_stats()


Timer unit: 1e-09 s

Total time: 11.4099 s
File: <ipython-input-28-246b588a091e>
Function: mean_rating_a at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def mean_rating_a(reviews):
     5         1      25097.0  25097.0      0.0      total_rating = 0
     6         1        300.0    300.0      0.0      count = 0
     7    126696 9968790512.0  78682.8     87.4      for _, row in reviews.iterrows():
     8    114602 1330030495.0  11605.6     11.7          if row['date'].year == 2010:
     9     12094  105727296.0   8742.1      0.9              total_rating += row['rating']
    10     12094    5346740.0    442.1      0.0              count += 1
    11         1       1743.0   1743.0      0.0      return total_rating / count if count > 0 else 0

Total time: 1.69361 s
File: <ipython-input-28-246b588a091e>
Function: mean_rating_b at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13        

In [30]:
#Да, можно. Используя метод loc

def mean_rating_b(reviews):
    reviews_2010 = reviews.loc[reviews['date'].dt.year == 2010]
    total_rating = reviews_2010['rating'].sum()
    count = reviews_2010['rating'].count()
    return total_rating / count if count > 0 else 0
start_time = time.time()
mean_b = mean_rating_b(reviews_2010)
print("Средний рейтинг по методу Б:", mean_b)
print("Время выполнения метода Б:", time.time() - start_time)

Средний рейтинг по методу Б: 4.4544402182900615
Время выполнения метода Б: 0.02646470069885254


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [28]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
%prun

In [30]:
from collections import defaultdict
from multiprocessing import Pool
from numba import jit

@jit(nopython=True)
def count_words(text, words):
    stats = defaultdict(int)
    for word in text.split():
        if word in words:
            stats[word] += 1
    return stats

def analyze_reviews(reviews, words):
    stats = defaultdict(int)
    with Pool() as p:
        results = p.starmap(count_words, [(review, words) for review in reviews])
        for result in results:
            for word, count in result.items():
                stats[word] += count
    return stats
%prun

4. Напишите несколько версий функции `MAPE` (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от среднего значения рейтинга по всем отзывам для этого рецепта. 
    1. Без использования векторизованных операций и методов массивов `numpy` и без использования `numba`
    2. Без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`
    3. С использованием векторизованных операций и методов массивов `numpy`, но без использования `numba`
    4. C использованием векторизованных операций и методов массивов `numpy` и `numba`
    
Измерьте время выполнения каждой из реализаций.

Замечание: удалите из выборки отзывы с нулевым рейтингом.


In [27]:
import pandas as pd

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += abs(actual_rating - rating) / rating
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [31]:
import timeit

def mape_v1(reviews):
    result = 0
    count = 0
    for recipe_id, recipe_group in reviews.groupby('recipe_id'):
        mean_rating = recipe_group['rating'].mean()
        for _, row in recipe_group.iterrows():
            if row['rating'] != 0:
                result += abs(row['rating'] - mean_rating) / mean_rating
                count += 1
    return result / count if count > 0 else 0

print(timeit.timeit(lambda: mape_v1(reviews), number=10, globals=globals())) # Для первой реализации

197.17042308500004


In [26]:
import pandas as pd
from numba import jit

@jit(nopython=True)
def mape_numba(actual_rating, rating):
    mape_sum = 0
    for i in range(len(actual_rating)):
        mape_sum += abs(actual_rating[i] - rating) / rating
    return mape_sum

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += mape_numba(actual_rating, rating)
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

In [33]:
import numpy as np
import timeit

def mape_v3(reviews):
    reviews = reviews[reviews['rating'] != 0]
    mean_ratings = reviews.groupby('recipe_id')['rating'].transform('mean')
    return np.mean(np.abs(reviews['rating'] - mean_ratings) / mean_ratings)

print(timeit.timeit(lambda: mape_v3(reviews), number=10, globals=globals())) # Для третьей реализации

0.35992541599989636


In [17]:
import pandas as pd
import numpy as np
from numba import jit

@jit(nopython=True)
def mape_numba(actual_rating, rating):
    return np.abs(actual_rating - rating) / rating

def MAPE(df: pd.DataFrame) -> float:
    # Удаляем строки с нулевым рейтингом
    df = df[df['rating'] != 0]
    
    # Группируем данные по рецепту
    grouped_data = df.groupby('recipe_id')
    
    # Вычисляем средний рейтинг для каждого рецепта
    mean_ratings = grouped_data['rating'].mean()
    
    # Вычисляем сумму абсолютных процентных отклонений для каждого рецепта
    mape_sum = 0
    for recipe_id, rating in mean_ratings.iteritems():
        recipe_data = df[df['recipe_id'] == recipe_id]
        actual_rating = recipe_data['rating'].values
        mape_sum += mape_numba(actual_rating, rating)
    
    # Вычисляем среднее абсолютное процентное отклонение для всех рецептов
    result = mape_sum.sum() / len(mean_ratings)
    
    return result

#### [версия 2]
* Уточнены формулировки задач 1, 3, 4